# Recipe2: Employer side Payroll Tax Offset 

This recipe is to introduce the usage of ` employer_payroll_offset` function from `payrolloffset.py`, when calling the function to conduct analysis upon employer side payroll tax reform

In [68]:
import taxcalcpayroll as tcp
from taxcalcpayroll.payrolloffset import employer_payroll_offset

import pandas as pd
import numpy as np

In [ ]:
recs = tcp.Records(data = 'puf.csv')
pol = tcp.Policy()
calc0 = tcp.Calculator(policy = pol, records = recs)

#### policy reform: increase the employer side FICA social security payroll tax rate to 0.072, for the year 2025 

In [70]:
reform1 = {'FICA_ss_trt_employer': {"2025": 0.072}}

### (1) total tax revenue after the reform (without offset)

In [ ]:
recs = tcp.Records(data = 'puf.csv')
pol1 = tcp.Policy()
pol1.implement_reform(reform1, print_warnings=True, raise_errors=True)
calc1 = tcp.Calculator(policy = pol1, records = recs)

calc1.advance_to_year(2025)
calc1.calc_all()
print("In 2025, combined tax revenue under reformed policy (without offset) is: ", calc1.weighted_total('combined') / 10**9)

In 2025, combined tax revenue under reformed policy (without offset) is:  $4053.78 Billion

### (2) total tax revenue after the reform, considering the offset

In [72]:
calc0.advance_to_year(2025)
calc0.calc_all()

### Function usage:
Input for offset function: baseline calculator (its policy and records object) and reform policy 

Output for offset function: returns a dataframe of output variables

In [ ]:
offdf = employer_payroll_offset(reform1, calc0, pol, recs, dump=False)    

print the result of calculated combined tax revenue

In [74]:
print("In 2025, combined tax revenue under reformed policy, after the offset, is: ", (offdf['combined'] * offdf['s006']).sum() / 10**9)

In 2025, combined tax revenue under reformed policy, after the offset, is:  4023.508250234729


In 2025, combined tax revenue under reformed policy, after the offset, is:  $4023.51 Billion

## Muti year usage

Comparison of total tax revenue calculation of baseline policy, reform policy (without offset), and reform policy considering offset

In [75]:
# setup some tool list to help printing result
dtable = pd.DataFrame([])
year_list = []
cur_list = []
reform_list = []
offset_list = []
base_ref_diff = []
ref_offset_diff = []
base_offset_diff = []

cur_sum = 0
reform_sum = 0
offset_sum = 0
base_ref_sum = 0
ref_offset_sum = 0
base_offset_sum = 0

In [ ]:
recs = tcp.Records(data = 'puf.csv')
pol = tcp.Policy()
calc0 = tcp.Calculator(policy = pol, records = recs)

recs = tcp.Records(data = 'puf.csv')
pol1 = tcp.Policy()
pol1.implement_reform(reform1, print_warnings=True, raise_errors=True)
calc1 = tcp.Calculator(policy = pol1, records = recs)

In [ ]:
# multi year 
for year in range(2024, 2034):   
    year_list.append(year)
    # baseline
    calc0.advance_to_year(year)
    calc0.calc_all()
    itax_rev0 = calc0.weighted_total('combined')
    itax_rev0 = (itax_rev0/10**9).round(2)
    cur_list.append(itax_rev0)
    cur_sum += itax_rev0
    
    # reform before offset
    calc1.advance_to_year(year)
    calc1.calc_all()
    itax_rev1 = calc1.weighted_total('combined')
    itax_rev1 = (itax_rev1/10**9).round(2)
    reform_list.append(itax_rev1)
    br_diff = itax_rev1 - itax_rev0
    base_ref_diff.append(br_diff)
    reform_sum += itax_rev1
    base_ref_sum += br_diff
    
    
    # offset
    offset_df = employer_payroll_offset(reform1, calc0, pol, recs, dump=False)
    itax_revbr = (offset_df['combined'] * offset_df['s006']).sum()
    itax_revbr = (itax_revbr/10**9).round(2)
    offset_list.append(itax_revbr)
    ro_diff = itax_revbr - itax_rev1
    ref_offset_diff.append(ro_diff)
    bo_diff = itax_revbr - itax_rev0
    base_offset_diff.append(bo_diff)
    offset_sum += itax_revbr
    ref_offset_sum += ro_diff
    base_offset_sum += bo_diff
    
    
year_list.append("10 year 2024-2033")  
cur_list.append(cur_sum)
reform_list.append(reform_sum)
base_ref_diff.append(base_ref_sum)
offset_list.append(offset_sum)
ref_offset_diff.append(ref_offset_sum)
base_offset_diff.append(base_offset_sum)


/Users/bodiyang/Desktop/taxcalc/Tax-Calculator/taxcalc/records.py:353: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.e00300 *= self.ADJ['INT{}'.format(year)][self.agi_bin].values
/Users/bodiyang/Desktop/taxcalc/Tax-Calculator/taxcalc/records.py:353: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.e00300 *= self.ADJ['INT{}'.format(year)][self.agi_bin].values
/Users/bodiyang/Desktop/taxcalc/Tax-Calculator/taxcalc/records.py:353: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

In [ ]:
df = pd.DataFrame({'Year': year_list, 'Baseline Tototal Revenue': cur_list, 'Reform Tototal Revenue (no offset)': reform_list,
                   'Reform Tototal Revenue with offset': offset_list, "Diff: Reform - Baseline": base_ref_diff,
                   'Diff: Offset - Reform': ref_offset_diff, "Diff: Offset - Baseline": base_offset_diff
                  })

In [ ]:
df = df.style.hide()

In [ ]:
display(df)